In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [82]:
hospital_df = pd.read_hdf("../../data/external/covid-hospitalizations.h5")
death_df = pd.read_hdf("../../data/external/new_deaths_per_million.h5")
vacc_age_df = pd.read_hdf("../../data/external/vaccinations-by-age-group.h5")
vacc_manu_df = pd.read_hdf("../../data/external/vaccinations-by-manufacturer.h5")
covid_df = pd.read_hdf("../../data/external/owid-covid-data.h5")

In [96]:
def get_count(df):
    try:
        df = df.groupby(["location"])[["date"]].count().rename(columns={"date": "count"}).reset_index()
    except:
        df = df.groupby(["entity"])[["date"]].count().rename(columns={"date": "count"}).reset_index().rename(columns={"entity":"location"})
    df_count = df.sort_values("count")
    return df_count

def get_boundary_count(boundary_count=200):
    below_boundary = []
    above_boundary = []
    below_boundary_country = []
    for df in [hospital_df, vacc_age_df, vacc_manu_df, covid_df]:
        below_boundary.append(len([i for i in get_count(df)["count"] if i < boundary_count]))
        above_boundary.append(len([i for i in get_count(df)["count"] if i > boundary_count]))
        below_boundary_country.append(get_count(df)[get_count(df)["count"] < boundary_count]["location"].tolist())
    return below_boundary, above_boundary, below_boundary_country

In [ ]:
# hospital_df = pd.read_hdf("../../data/external/owid-covid-data.h5")
# # df_count = hospital_df.groupby(["location"])[["iso_code"]].count().rename(columns={"iso_code": "count"}).reset_index()
# # df_count = df_count.sort_values("count")
# get_count(hospital_df )

# plt.figure(figsize=(4,45))
# plt.style.use('ggplot')

# plt.barh(df_count["location"].tolist(), df_count["count"].tolist())
# plt.title("Data Counts for Each Country")
# plt.ylabel("country")
# plt.xlabel("count")
# plt.show()

In [106]:
boundary = 300
df = pd.DataFrame.from_dict({"less than "+str(boundry)+" data": get_boundary_count(boundary)[0],
                             "more than "+str(boundry)+" data": get_boundary_count(boundary)[1],
                             "country name with data < "+str(boundry): get_boundary_count(boundary)[2]})
df.index = ["covid-hospitalizations", "vaccinations-by-age-group",
            "vaccinations-by-manufacturer", "owid-covid-data"]
print("Countries with less than "+str(boundary)+" data entries will not be used: ", 
      set(sum(get_boundary_count(boundary)[2], [])), "\n")
print("Data Counts of Countries:")
df

Countries with less than 300 data entries will not be used:  {'Western Sahara', 'Singapore', 'Canada', 'Liechtenstein', 'Finland', 'Iceland', 'Russia', 'Sweden'} 

Data Counts of Countries:


,less than 300 data,more than 300 data,country name with data < 300
covid-hospitalizations,3,48,"[Russia, Liechtenstein, Singapore]"
vaccinations-by-age-group,0,38,[]
vaccinations-by-manufacturer,5,39,"[Iceland, Liechtenstein, Sweden, Finland, Canada]"
owid-covid-data,1,247,[Western Sahara]
